# Student Contribution

An awesome variation that includes a tool to make a booking! Thank you! -- Ed

# Project - Airline AI Assistant with Booking Tool

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import pandas as pd
import google.generativeai as genai

In [2]:
load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
MODEL = "gemini-1.5-flash"
genai.configure(api_key=google_api_key)

In [3]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

## Tools

* Price and Booking

In [4]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [5]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city, Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'.Be accurate, if you dont have price information for a particular ,destination say so.",
    "parameters": {
        "type":"OBJECT",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }
        },
        "required": ["destination_city"]
    }
}

booking_function = {
    "name": "book_ticket",
    "description": "Book a ticket to a destination city",
    "parameters": {
        "type":"OBJECT",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city the customer wants to travel to"
            },
            "num_tickets": {
                "type": "integer",
                "description": "The number of tickets to book"
            },
            "ticket_class": {
                "type": "string",
                "description": "The class of the ticket (e.g., economy, business)"
            },
            "mail_address": {
                "type": "string",
                "description": "Mail address to send the ticket to"
            }
        },
        "required": ["destination_city", "num_tickets", "ticket_class", "mail_address"]
    }
}


In [6]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": booking_function}]

## Creating Application

In [7]:
# Creating  ID for each booking
# Produces IDs like 'BK-000001', 'BK-000002'

booking_counter = 0

def generate_unique_booking_id():
    global booking_counter
    booking_counter += 1
    return f"BK-{booking_counter:06d}"  


In [8]:
bookingDB = pd.DataFrame()

In [9]:
def handle_tool_call_gemini(function_call):
    if function_call.name == "get_ticket_price":
        args = function_call.args
        destination_city = args.get('destination_city')
        price = get_ticket_price(destination_city)
        if price == "Unknown":
            return f"Sorry, we don't fly to {destination_city}. Available destinations are: {', '.join(ticket_prices.keys())}"
        return f"The price for {destination_city} is {price}"
    
    elif function_call.name == "book_ticket":
        args = function_call.args
        destination_city = args.get('destination_city')
        num_tickets = args.get('num_tickets')
        ticket_class = args.get('ticket_class')
        mail_address = args.get('mail_address')
        
        # Get the ticket price
        price_str = int(get_ticket_price(destination_city))
        if price_str == "Unknown":
            return f"Sorry, we don't fly to {destination_city}. Available destinations are: {', '.join(ticket_prices.keys())}"
        price= int(price_str.replace("$", ""))
        total_price = price * num_tickets
        
        # Generate booking ID and save to DB (keeping existing logic)
        booked_ID = generate_unique_booking_id()
        data = {
            "booking_id": [booked_ID],
            "mail_address": [mail_address],
            "destination_city": [destination_city],
            "num_tickets": [num_tickets],
            "ticket_class": [ticket_class],
            "total_price": [total_price],
        }
        booking_temp = pd.DataFrame(data)
        global bookingDB
        bookingDB = bookingDB._append(booking_temp)
        bookingDB.to_csv('bookingDB.csv', index=False)
        
        return f"Booking confirmed! Booking ID: {booked_ID} for {num_tickets} {ticket_class} ticket(s) to {destination_city}. Total price: ${total_price}"

In [10]:
def chat(message, history):
    # Initialize Gemini model
    model = genai.GenerativeModel(MODEL)
    chat = model.start_chat(history=[])  
    # Add previous messages to chat history
    for human, assistant in history:
        chat.send_message(human)
        chat.send_message(assistant)
    
    # Send current message with tools
    response = chat.send_message(
        message,
        generation_config={
            "temperature": 0.7
        },
        tools=[
            {
                "function_declarations": [price_function, booking_function]
            }
        ]
    )
    
    # Check if function call is needed
    if hasattr(response, 'candidates') and response.candidates[0].content.parts[0].function_call:
        function_call = response.candidates[0].content.parts[0].function_call
        tool_response = handle_tool_call_gemini(function_call)
        
        # Send follow-up with tool response
        response = chat.send_message(
            f"Tool response: {tool_response}",
            generation_config={"temperature": 0.7}
        )
    
    return response.text

In [11]:
gr.ChatInterface(fn=chat).launch()

c:\Users\anura\anaconda3\envs\llm_engineering\Lib\site-packages\gradio\components\chatbot.py:231: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\anura\anaconda3\envs\llm_engineering\Lib\site-packages\gradio\queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anura\anaconda3\envs\llm_engineering\Lib\site-packages\gradio\route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anura\anaconda3\envs\llm_engineering\Lib\site-packages\gradio\blocks.py", line 2015, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anura\anaconda3\envs\llm_engineering\Lib\site-packages\gradio\blocks.py", line 1560, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\anura\anaconda3\envs\llm_engineering\Lib\site-packages\gradio\utils.py", line 832, in async_wrapper

## Enjoy, Happy Learning Thanks to Ed!